# Basic scraping for Guardian discussions u

In [1]:
pwd

'/home/gregmcshane/GUARDIAN'

In [4]:
with open('guardian comments scraper.ipynb','r') as fp:
    nb = fp.read()

In [9]:
import re

pp = re.compile("(https://www.the.*?)'")
lks = pp.findall(nb)

with open('links.txt','w') as fp:
    fp.write('\n'.join(lks))

In [1]:
import re 
import requests
import pickle
import json

In [11]:
article_url = 'https://www.theguardian.com/commentisfree/2020/aug/19/ditch-the-algorithm-generation-students-a-levels-politics'

In [3]:
key = my_keys.keys['guardian']

In [12]:
#scrape the page and get the short url
short_url_pp =  re.compile('"shortUrlId":"(.*?)"')

r = requests.get(article_url)
mm = short_url_pp.search(r.text)
short_url = mm.group(1)

#the short url allows us to access the discussion
comments_url = 'https://api.nextgen.guardianapps.co.uk/discussion/%s.json'%short_url
params = {'orderBy' : 'newest',
          'pageSize' : 25,
          'displayThreaded' : 'true',
          'commentsClosed' : 'false',
          'page' : '',
          'maxResponses' : 50
         }

json_data =  []
start_page  = 1
last_page = None


for num_page in range(start_page, 10**4):
    print('getting comments page no.', num_page)
    params['page'] = num_page
    r = requests.get( comments_url, params=params)
    json_data.append( r.text)
    #The last page is a field in the json object
    #I don't need to read this as json I could do a regexp
    if num_page == start_page:
        json_dict = json.loads(r.text)
        last_page = json_dict['lastPage']
        print('there are %d pages'%last_page)
    if num_page == last_page : break
        

fn = '%s.pkl'%short_url.split('/')[-1]
print('archiving to %s'%fn)
with open(fn ,'wb') as fp:
    pickle.dump(json_data,fp)
    


getting comments page no. 1
there are 8 pages
getting comments page no. 2
getting comments page no. 3
getting comments page no. 4
getting comments page no. 5
getting comments page no. 6
getting comments page no. 7
getting comments page no. 8
archiving to etenk.pkl


In [2]:
json_data = pickle.load(open('dpb6e.pkl','rb') ) #jenkins

archiving to dqhaa.pkl #jenkins

archiving to evdjz.pkl #big cummings

e9j7e.pkl # bragg

eghyd.pkl # migrants

SyntaxError: invalid syntax (<ipython-input-2-7a525c39f9dc>, line 3)

In [3]:
json_data = pickle.load(open('etenk.pkl','rb') ) #jenkins


## remember each page of comments has been stored separately

In [4]:
dd = json.loads(json_data[0])
len(json_data)

8

In [8]:
xx = json.loads(json_data[0])
xx.keys()

dict_keys(['postedCommentHtml', 'commentsHtml', 'commentCount', 'refreshStatus', 'lastPage', 'paginationHtml'])

In [9]:
print( xx['commentsHtml'][:100])



<div class="d-discussion d-discussion--recommendations-open u-cf"
     data-read-only="false" >

 


In [5]:
pa = re.compile('data-comment-author="(.*?)"')
pid = re.compile('data-comment-author-id="(\d+)')

auths = []
for xx in json_data:
    hh = json.loads(xx)['commentsHtml']
    auths.extend(list( zip(pa.findall(hh) , pid.findall(hh)) ))

auths = list(set(auths))
auths.sort(key=lambda x : x[0].lower())


In [6]:
auths = dict(auths)

In [7]:
class Comment():
    pass

from bs4 import BeautifulSoup

def html2comments(x):
    tt = json.loads(x)
    soup = BeautifulSoup(tt['commentsHtml'],  "lxml")
    raw = [ comment.text for comment in  
            soup.findAll('div', {'class' : "d-comment__body"}) ]
    
    metadata = soup.findAll('span', {'class' : "d-comment__author"})
    
    auth = [x.text.upper().strip() for x in metadata]
    auth_id = [ x.find('a')['href'] for x in metadata]
    
    return [ '##{}\n{}\n\n{}'.format(a,b,c.strip())  
                                 for a,b,c in zip(auth, auth_id, raw)]
     
all_texts = [ html2comments(page) for page in json_data[:] ]



from itertools import chain
all_comments = list(chain.from_iterable(all_texts))
    

with open('b7jtb.pkl','rb') as fp:
    json_data = pickle.load(fp)

In [8]:
A = [x.split('\n')[0] for x in all_comments]
from collections import Counter
AN = Counter(A)

In [9]:
sorted([(x,y) for x,y in AN.items() if y > 4], key=lambda x : -x[1]) 

[('##BJERKLEY', 45),
 ('##FATBOB', 41),
 ('##AJVC1991', 26),
 ('##SPIKE501', 20),
 ('##SKEPTICHAPPY', 13),
 ('##OLDWEASE', 13),
 ('##EASTONHALL33', 11),
 ('##JDRMCR', 10),
 ('##BREXITSBANE', 10),
 ('##HUGHBRYANT', 10),
 ('##NICCOLOFRANZORWELL', 10),
 ('##SNOWYJOHN', 9),
 ('##SWAN17', 9),
 ('##GEOFF001', 8),
 ('##FALLOWFIELD', 8),
 ('##FROMAGEBLEU', 7),
 ('##ALIEN98', 7),
 ('##FOSTER6THE6IMPOSTER6', 6),
 ('##CITIZEN_OF_EUROPE', 6),
 ('##TONDA99', 6),
 ('##SJAMES74', 6),
 ('##SEELIFEDIFFERENTLY', 6),
 ('##RHUBARBSONOFNECTAN', 6),
 ('##SHAKINGHEADSLOWLY', 5),
 ('##MISSILEMAN', 5),
 ('##ABBYLOX', 5),
 ('##WHATABOUTTHEFORESTS', 5),
 ('##CAVC66', 5),
 ('##SCUBADOC', 5),
 ('##KAVANGH', 5)]

In [10]:
all_comments.sort(key=lambda x : -len(x) )

In [11]:
print( '\n\n'.join(all_comments[:20]) )

##RJI777
https://profile.theguardian.com/user/id/102379177

The author might well be considered an expert on the subject of politics, but when it comes to "algorithms", "predictive modelling" and the recent "statistical" catastrophes, the article suggests that she lacks sufficient understand of the issues outwith her primary area of expertise.
So, brief as it may be, perhaps the following might help:
First: 1/ Algorithms, etc., apart, we lack the capacity to predict, with any certainty or consistency, the future in terms of an actual reality and all within which it is comprised;
Second: 2/ Even if we could conquer 1/, our new found super powers would soon prove to be very restricted in terms of their use. This is because "predictive modelling" and the likes attempt to deal with rather large complex (sub) systems e.g., the UK, etc. Simply put, a key characteristic of complex systems is that they behave in all manner of counterintuitive ways which makes them, well, unpredictable;
Third: 

In [18]:
#https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

import nltk
nltk.download('vader_lexicon')

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/gregmcshane/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [22]:
a = '''My views haven't changed. Leaving the EU will be of no benefit to anyone except those who hold money and power along with certain corporates and a smidgen of medium to small businesses.
How else can it be when you are shutting shop to the most developed nations, the richest nations in the world, who are either in the EU or have market agreements with the EU. Even the emerging markets of China and India along with other developing economies are aspiring to develop trading agreements with the EU, the second largest economy in the world. Any deal with the UK will be secondary and will depend upon their agreement with the EU.
Boris Johnson once said "f**k business". I take this as meaning there is something in it for them and they don't really care if businesses find it difficulA
t to cope.
I can see ministers and conservative MPs queuing to fill their pockets by acting as advisors for potential trade deals whilst the rest of the country are left struggling trying to rebuild their lives, especially following the pandemic. The same ministers will probably be looking to lower all kinds of standards from food and welfare to safety at work to improve their opportunities.
When it comes to brinkmanship, I can only view this as getting away with whatever they can. I do not believe it is anything to do with negotiating an agreement that will be to the benefit of the UK or the EU as a whole, I don't believe our ministers are capable of doing that.'''

In [24]:
sid.polarity_scores(a)
sid.sentiment_valence(a)

TypeError: sentiment_valence() missing 4 required positional arguments: 'sentitext', 'item', 'i', and 'sentiments'

In [14]:
target = 'decolon'
target = 'science'
target = ' math'

xx = [x for x in all_comments if target.lower() in x.lower() and  len(x) > 20]
yy = [x for x in all_comments if '#pin' in x.lower() and 2000 > len(x) > 20]

#print('\n\n'.join( ['\n\n'.join(x) for x in zip(xx,yy)])  )


print( '\n\n'.join(xx).replace(target, '**'+ target.upper() +'**') )
#print( '\n\n'.join(xx) )


##CITIZEN_OF_EUROPE
https://profile.theguardian.com/user/id/101220304

The problem is not the use of algorithms per se, but the judgement of whether a particular algorithm is fit for the purpose to which it is being put. Algorithms have the advantage of giving unambiguous answers and in one sense are objective, in that once put in operation the user cannot influence the outcome.
The problem in this case is that the algorithm is designed to maintain a consistent standard in terms of average and spread from one year to the next. Because it is based on the statistics that it is intended to maintain, it is effectively inevitable that it will achieve its purpose, which is of course why it is utterly useless for giving individual pupils a fair assessment - that would allow unique achievement to undermine the principle of maintaining the standard.
There is nothing new in this in education, although the process has got steadily worse over the last ten years - since Michael Gove linked individu

36